In [1]:
!pip install segmentation-models-pytorch > /dev/null

In [2]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class WaterBodiesDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_list = [f for f in os.listdir(os.path.join(data_dir, 'Image')) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_name = self.image_list[idx]
        image_path = os.path.join(self.data_dir, 'Image', image_name)
        mask_name = os.path.splitext(image_name)[0] + '.png'
        mask_path = os.path.join(self.data_dir, 'Mask', mask_name)

        image = Image.open(image_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [4]:
data_dir = '/kaggle/input/flood-area-segmentation'
dataset = WaterBodiesDataset(data_dir, transform=data_transforms)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [5]:
ENCODER='resnet18'
WEIGHTS='imagenet'

In [6]:
class SegmentationModel(nn.Module):
    def __init__(self):
        super(SegmentationModel,self).__init__()

        self.arc=smp.Unet(
            encoder_name=ENCODER,
            encoder_weights=WEIGHTS,
            in_channels=3,
            classes=1,
            activation=None
        )
    def forward(self,images,masks=None):
        logits=self.arc(images)

        if masks!=None:
            loss1=DiceLoss(mode='binary')(logits,masks)
            loss2=nn.BCEWithLogitsLoss()(logits,masks)
            return logits,loss1,loss2
        return logits

In [ ]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()

# set the device to use for computations
device = torch.device("cuda" if use_cuda else "cpu")

# define the model
model = SegmentationModel()
model.to(device)

# define the loss function
criterion = nn.BCEWithLogitsLoss()

# define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

# define the number of epochs to train for
num_epochs = 500

# create a SummaryWriter to log training progress to TensorBoard
writer = SummaryWriter()

highest_accuracy = 0
model_path = 'resnet_18_final.pt'

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    # create a progress bar using tqdm
    pbar = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, data in pbar:
        # get the inputs and labels and move them to the GPU
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # compute the accuracy
        predicted = outputs > 0.5
        total += labels.numel()
        correct += (predicted == labels).sum().item()

        # backward pass
        loss.backward()
        optimizer.step()

        # update the progress bar and print statistics
        running_loss += loss.item()
        pbar.set_description(f'Epoch: {epoch + 1}, Loss: {running_loss / (i + 1):.3f}, Accuracy: {correct / total:.3f}')

    # log the loss and accuracy to TensorBoard
    writer.add_scalar('Loss/train', running_loss / len(dataloader), epoch)
    writer.add_scalar('Accuracy/train', correct / total, epoch)
    
    if correct / total > highest_accuracy:
        highest_accuracy = correct / total
        torch.save(model.state_dict(), model_path)

print('Finished Training')


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 196MB/s] 
Epoch: 65, Loss: 0.235, Accuracy: 0.870:  71%|███████   | 52/73 [00:13<00:05,  3.59it/s]

In [ ]:
!pip install gdown  
!gdown --id 16sRHSusyhK3SqUggW6ZTvEYVIUHYAl7J

In [ ]:
from zipfile import ZipFile
with ZipFile('dataset.zip', 'r') as zipObj:
   zipObj.extractall()

In [ ]:
class TestDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_list = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_name = self.image_list[idx]
        image_path = os.path.join(self.data_dir, image_name)

        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image_name, image

In [ ]:
# Test dataset path: /kaggle/working/dataset/flooded
test_data_dir = '/kaggle/working/dataset/flooded'
test_dataset = TestDataset(test_data_dir, transform=data_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# create the output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# set the model to evaluation mode
model.eval()

with torch.no_grad():
    for i, data in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        # get the image name and input
        image_name, inputs = data
        inputs = inputs.to(device)

        # forward pass
        outputs = model(inputs)
        predicted = outputs > 0.5

        # save the predicted mask
        mask_name = os.path.join('output', image_name[0])
        Image.fromarray(predicted.squeeze().cpu().numpy()).save(mask_name)

In [ ]:
def apply_mask(image, mask):
    # convert the image and mask to numpy arrays
    image = np.array(image)
    mask = np.array(mask)

    # apply the mask to each channel of the image

    masked_image = image.copy()
    masked_image[0, :, :] = image[0, :, :].copy() * mask
    masked_image[1, :, :] = image[1, :, :].copy() * mask
    masked_image[2, :, :] = image[2, :, :].copy() * mask

    # transpose the masked image to have shape (height, width, channels)
    masked_image = np.transpose(masked_image, (1, 2, 0))

    # convert the masked image to uint8
    masked_image = masked_image.astype(np.uint8)

    return masked_image

# create the mask_output directory if it doesn't exist
os.makedirs('mask_output', exist_ok=True)

# create a new FloodTestDataset for the test data
test_data_dir = 'dataset/flooded'
test_dataset = TestDataset(test_data_dir, transform=data_transforms)

# iterate over the test data
for i in tqdm(range(len(test_dataset)), total=len(test_dataset)):
    # get the image name and image
    image_name, image = test_dataset[i]

    # load the predicted mask
    mask_name = os.path.join('output', image_name)
    mask = Image.open(mask_name).convert('L')

    # apply the mask to the image
    masked_image = apply_mask(image, mask)

    # save the masked image
    masked_image_name = os.path.join('mask_output', image_name)
    Image.fromarray(masked_image).save(masked_image_name)


In [ ]:
# Show the original image & the masked image (from mask_output folder)

img_name = '1.jpg'
img_path = os.path.join('dataset/flooded/', img_name)
masked_img_path = os.path.join('mask_output/', img_name)

img = Image.open(img_path)
masked_img = Image.open(masked_img_path)

fig, ax = plt.subplots(1, 2, figsize=(10, 15))
ax[0].imshow(img)
ax[1].imshow(masked_img)
plt.show()

In [ ]:
# Pickle the model
model_path = 'resnet18 - 93%.pt'
torch.save(model.state_dict(), model_path)

In [ ]:
model.load_state_dict(torch.load(model_path))

In [ ]:
model.to(device)

# define the loss function
criterion = nn.BCEWithLogitsLoss()

# define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# define the number of epochs to train for
num_epochs = 100

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    # create a progress bar using tqdm
    pbar = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, data in pbar:
        # get the inputs and labels and move them to the GPU
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # compute the accuracy
        predicted = outputs > 0.5
        total += labels.numel()
        correct += (predicted == labels).sum().item()

        # backward pass
        loss.backward()
        optimizer.step()

        # update the progress bar and print statistics
        running_loss += loss.item()
        pbar.set_description(f'Epoch: {epoch + 1}, Loss: {running_loss / (i + 1):.3f}, Accuracy: {correct / total:.3f}')

    # log the loss and accuracy to TensorBoard
    writer.add_scalar('Loss/train', running_loss / len(dataloader), epoch)
    writer.add_scalar('Accuracy/train', correct / total, epoch)

print('Finished Training')